

#### **PROJECT INTRODUCTION**

I am  are a cybersecurity analyst for yummyrecipesforme.com, a website that sells recipes and cookbooks. A former employee has decided to lure users to a fake website with malware. 

The former employee/ hacker executed a **brute force attack** to gain access to the web host. They repeatedly entered several known default passwords for the **administrative account** until they correctly guessed the right one. After they obtained the **login credentials**, they were able to access the **admin panel** and **change the website’s source code**. They embedded a **javascript function** in the source code that prompted visitors to download and run a file upon visiting the website. After embedding the malware, the hacker changed the password to the administrative account. When customers download the file, they are redirected to a fake version of the website that contains the **malware**. 

Several hours after the attack, multiple customers emailed yummyrecipesforme’s helpdesk. They complained that the company’s website had prompted them to download a file to access free recipes. The customers claimed that, after running the file, the address of the website changed and their personal computers began running more slowly. 

In response to this incident, the website owner tries to log in to the admin panel but is unable to, so they reach out to the website hosting provider. You and other cybersecurity analysts are tasked with investigating this security event.

To address the incident, you create a **sandbox environment** to observe the suspicious website behavior. You run the **network protocol analyzer tcpdump**, then type in the URL for the website, yummyrecipesforme.com. As soon as the website loads, you are prompted to download an executable file to update your browser. You accept the download and allow the file to run. You then observe that your browser redirects you to a different URL, greatrecipesforme.com, which contains the malware.  

The logs show the following process:

The browser initiates a **DNS request**: It requests the **IP address** of the yummyrecipesforme.com URL from the **DNS server**.

The **DNS** replies with the correct IP address. 

The browser initiates an **HTTP request**: It requests the yummyrecipesforme.com webpage using the **IP address** sent by the DNS server.

The browser initiates the download of the **malware**.

The browser initiates a **DNS request** for greatrecipesforme.com.

The DNS server responds with the **IP address** for greatrecipesforme.com.

The browser initiates an **HTTP** request to the IP address for greatrecipesforme.com.

**A senior analyst confirms that the website was compromised. The analyst checks the source code for the website. They notice that javascript code had been added to prompt website visitors to download an executable file. Analysis of the downloaded file found a script that redirects the visitors’ browsers from yummyrecipesforme.com to greatrecipesforme.com.** 

The cybersecurity team reports that the web server was impacted by a **brute force attack**. The disgruntled hacker was able to guess the password easily because the admin password was still set to the default password. Additionally, there were no controls in place to prevent a brute force attack. 

Your job is to document the incident in detail, including identifying the network protocols used to establish the connection between the user and the website.  You should also recommend a security action to take to prevent brute force attacks in the future.

 #### **Analysis of tcpdump Traffic Log**

This analysis using `tcpdump` identifies the events in the traffic log, emphasizing the behavior of the brute force attack and subsequent malicious redirection.




 **1. Initial DNS Resolution for `yummyrecipesforme.com`**

```plaintext
14:18:32.192571 IP your.machine.52444 > dns.google.domain: 35084+ A? yummyrecipesforme.com. (24)
14:18:32.204388 IP dns.google.domain > your.machine.52444: 35084 1/0/0 A 203.0.113.22 (40)

Event: The client initiates a DNS request using port 52444 to resolve the domain name yummyrecipesforme.com to an IP address.
Result: The DNS server responds with the IP address 203.0.113.22, allowing the client to proceed with **HTTP communication.




**2. Establishing a TCP Connection to yummyrecipesforme.com**

14:18:36.786501 IP your.machine.36086 > yummyrecipesforme.com.http: Flags [S], seq 2873951608, win 65495, options [mss 65495,sackOK,TS val 3302576859 ecr 0,nop,wscale 7], length 0
14:18:36.786517 IP yummyrecipesforme.com.http > your.machine.36086: Flags [S.], seq 3984334959, ack 2873951609, win 65483, options [mss 65495,sackOK,TS val 3302576859 ecr 3302576859,nop,wscale 7], length 0
14:18:36.786529 IP your.machine.36086 > yummyrecipesforme.com.http: Flags [.] ack 1, win 512, options [nop,nop,TS val 3302576859 ecr 3302576859], length 0

Event: A three-way TCP handshake begins:
[S]: The client (your.machine.36086) sends a synchronization (SYN) packet to yummyrecipesforme.com.
[S.]: The server responds with a SYN-ACK packet, confirming it can receive communication.
[.]: The client acknowledges (ACK), completing the handshake.



**3. HTTP GET Request to yummyrecipesforme.com**

14:18:36.786589 IP your.machine.36086 > yummyrecipesforme.com.http: Flags [P.], seq 1:74, ack 1, win 512, options [nop,nop,TS val 3302576859 ecr 3302576859], length 73: HTTP: GET / HTTP/1.1
14:18:36.786595 IP yummyrecipesforme.com.http > your.machine.36086: Flags [.] ack 74, win 512, options [nop,nop,TS val 3302576859 ecr 3302576859], length 0

Event: The client sends an HTTP GET request to yummyrecipesforme.com using port 80.
Detail: The request length is 73 bytes, representing headers and other metadata needed to request the webpage.
Server Response: The server acknowledges receipt of the HTTP request.






**4. DNS Resolution for greatrecipesforme.com**

14:20:32.192571 IP your.machine.52444 > dns.google.domain: 21899+ A? greatrecipesforme.com. (24)
14:20:32.204388 IP dns.google.domain > your.machine.52444: 21899 1/0/0 A 192.0.2.17 (40)

Event: After executing the malicious JavaScript embedded in yummyrecipesforme.com, the client initiates a DNS request for greatrecipesforme.com.

Result: The DNS server resolves greatrecipesforme.com to 192.0.2.17, redirecting the client to the fake website.




**5. Establishing a TCP Connection to greatrecipesforme.com**

14:25:29.576493 IP your.machine.56378 > greatrecipesforme.com.http: Flags [S], seq 1020702883, win 65495, options [mss 65495,sackOK,TS val 3302989649 ecr 0,nop,wscale 7], length 0
14:25:29.576510 IP greatrecipesforme.com.http > your.machine.56378: Flags [S.], seq 1993648018, ack 1020702884, win 65483, options [mss 65495,sackOK,TS val 3302989649 ecr 3302989649,nop,wscale 7], length 0
14:25:29.576524 IP your.machine.56378 > greatrecipesforme.com.http: Flags [.] ack 1, win 512, options [nop,nop,TS val 3302989649 ecr 3302989649], length 0

Event: The client establishes a new TCP connection to the fake website (greatrecipesforme.com) using port 56378.
Three-Way Handshake:
[S]: Client sends a SYN packet to the fake server.
[S.]: Fake server responds with SYN-ACK.
[.]: Client acknowledges and establishes the connection.





6. HTTP GET Request to greatrecipesforme.com

14:25:29.576590 IP your.machine.56378 > greatrecipesforme.com.http: Flags [P.], seq 1:74, ack 1, win 512, options [nop,nop,TS val 3302989649 ecr 3302989649], length 73: HTTP: GET / HTTP/1.1
14:25:29.576597 IP greatrecipesforme.com.http > your.machine.56378: Flags [.] ack 74, win 512, options [nop,nop,TS val 3302989649 ecr 3302989649], length 0

Event: The client sends an HTTP GET request to greatrecipesforme.com using port 80, completing the redirection initiated by the malicious file.

Detail: The malicious fake website responds, serving malware or other malicious content.




#### **Cybersecurity Incident Report**

**Section 1: Identify the Network Protocol Involved**

**Network Protocol: HTTP (Hypertext Transfer Protocol)**

**HTTP** was used to facilitate communication between the user's browser and the web server. It enabled the transmission of website data, including the malicious script embedded by the attacker.

**Additional Protocols Identified:**

**DNS (Domain Name System)**: Used to resolve the domain names yummyrecipesforme.com and greatrecipesforme.com into their respective IP addresses.

**TCP (Transmission Control Protocol)**: Used for establishing and maintaining connections during HTTP requests.


**Section 2: Document the Incident**

**Incident Summary:** 
A former employee executed a **brute force attack** to guess the default administrative password for the web server. After gaining access to the admin panel, the attacker embedded a malicious JavaScript function into the website's source code. This function prompted users to download a file disguised as a browser update.

**Malicious Behavior:**

The downloaded file contained a script that redirected users from yummyrecipesforme.com to the fake website greatrecipesforme.com, which hosted malware.
The fake website caused users' computers to run slowly, likely due to the malware executing on their systems.


**Investigation Findings:**
DNS Resolution:
The browser sent a DNS request for yummyrecipesforme.com and received a response with the correct IP address.
HTTP Requests:
The browser initiated an HTTP request to yummyrecipesforme.com, triggering the malicious JavaScript to prompt a file download.
Redirection:
After running the file, the browser sent another DNS request for greatrecipesforme.com and connected to its IP address using HTTP.


**Logs Confirm:**
The malicious JavaScript embedded in the web server redirected traffic to the spoofed website, and the fake website hosted malware.


**Section 3: Recommend One Security Measure**


**Recommended Measure:** Implement Multi-Factor Authentication (MFA) and Enforce Strong Password Policies

**Details:**
Replace the default password with a strong, unique password following best practices (e.g., 12+ characters, mixed case, numbers, special characters).
Implement MFA for administrative accounts to prevent unauthorized access, even if passwords are compromised.
Enable account lockout policies to block login attempts after multiple failed attempts, mitigating brute force attacks.


**Additional Measures:**

Regularly audit and update security controls.
Train employees on cybersecurity awareness to recognize malicious behavior.


**Conclusion**

The investigation confirmed that a brute force attack exploited a default password to compromise the web server. The attacker embedded a malicious script, redirecting users to a fake website hosting malware. Implementing MFA and enforcing strong password policies will significantly reduce the risk of similar attacks in the future.